In [52]:
import pandas as pd
import re
import json
import glob
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Define range for file, friends change this help me download when you're free lel
start_int = 0
end_int = 100

In [26]:
# Combine CSV, generate output

files = glob.glob("Clean_data/*.csv")

csv = []

for file in files:
    df = pd.read_csv(file, index_col=None, header=0)
    csv.append(df)

df = pd.concat(csv, axis=0, ignore_index=True)

In [27]:
print(df.shape, "\n",df.columns)

(195933, 27) 
 Index(['Unnamed: 0', 'backers_count', 'blurb', 'category',
       'converted_pledged_amount', 'country', 'created_at', 'current_currency',
       'deadline', 'fx_rate', 'goal', 'id', 'launched_at', 'location', 'name',
       'pledged', 'profile', 'slug', 'source_url', 'spotlight', 'staff_pick',
       'state', 'state_changed_at', 'static_usd_rate', 'urls', 'usd_pledged',
       'usd_type'],
      dtype='object')


In [34]:
#urls = df.urls
#ids = df.id

#urls_ = list(map(lambda url:json.loads(url)["web"]["project"],urls))
#urls = urls_

#get url
url = df[df.state == "successful"].urls.reset_index().urls[1]
url = json.loads(url)
url = url["web"]["project"]
url

'https://www.kickstarter.com/projects/toadthewetsprocket/new-studio-album-from-toad-the-wet-sprocket?ref=discovery_category_newest'

In [54]:
#path = "Webdriver/chromedriver.exe"
#driver = webdriver.Chrome(path)
#driver = webdriver.Chrome(executable_path='/path/to/chromedriver')

#from webdriver_manager.chrome import ChromeDriverManager
#driver = webdriver.Chrome(ChromeDriverManager().install())


from selenium.webdriver.support import ui
from selenium.common.exceptions import TimeoutException
driver = webdriver.Chrome()

comments = []

#for i in tqdm(range(start_int,end_int)):
    #driver.get(urls[i])
driver.get(url)


try:
    wait = ui.WebDriverWait(driver, 10)
    wait.until(lambda driver: driver.find_element_by_xpath('//div[@id="comments-emoji"]')).click()
    #driver.find_element_by_id('comments-emoji').click()
except TimeoutException:
    pass
    

try: 
    comments_raw = WebDriverWait(driver,10).until(
    EC.presence_of_element_located((By.XPATH,"//div[@id='react-project-comments']"))
)
    #for comment in comments_raw:
    comments.append(comments_raw.text)

except TimeoutException:
    pass

driver.quit()

In [55]:
comments

[]

In [ ]:
df_export = pd.DataFrame({"index":list(range(start_int,end_int)),
                         "comments":comments}, index = None)
filename = "Comments_" + str(start_int) + "-" + str(end_int)
df_export.to_csv("Output/Comments/{}.csv".format(filename))